In [35]:
import re

def parse_schedule(text):
    schedule = {}
    weekdays = ['SEGUNDA-FEIRA', 'TERCA-FEIRA', 'QUARTA-FEIRA', 'QUINTA-FEIRA', 'SEXTA-FEIRA', 'SABADO', 'DOMINGO']
    lines = text.splitlines()
    current_day = None

    # create dictionary with keys as weekdays and values as a list with every index as one class time entry
    #{'SEGUNDA-FEIRA': ['10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula', '10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula'], 'QUINTA-FEIRA': ['10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula', '10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula'], 'SEXTA-FEIRA': ['17:10 - 18:00    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula', '18:00 - 18:50    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula']}
    for line in lines:
        if line in weekdays:
            current_day = line
            schedule[current_day] = []
        elif current_day is not None:
            schedule[current_day] += line.strip().split('\n')


    # the dictionary schedule will be updated in a way that the keys will continue to be the days of the week but
    # values will be a list of dictionaries, every dictionary being a class time entry
    # and this dictionary will have keys: time,subject,location
    for key,value in schedule.items():
        lst = []
        for entry in value:
            parts = re.split(r'\s{2,}', entry)
            time = parts[0]
            subject = parts[1].split('-')[-1]
            location = parts[2].split('-')[0]
            lst.append({'time':time,
                        'subject': subject,
                        'location' :location})
        schedule[key]= lst
    


    return schedule


text = """
ÑÃO FINROMADO
SEGUNDA-FEIRA
10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula
10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 343 - Sala de aula
QUINTA-FEIRA
10:00 - 10:50    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula
10:50 - 11:40    22000303 - M1 - PROJETO DE COMPILADORES    [ANG] 235 - Sala de Aula
SEXTA-FEIRA
17:10 - 18:00    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula
18:00 - 18:50    22000306 - M1 - TRABALHO DE CONCLUSÃO DE CURSO II    [ANG] 342 - Sala de aula"""

schedule = parse_schedule(text)
print(schedule)

{'SEGUNDA-FEIRA': [{'time': '10:00 - 10:50', 'subject': ' PROJETO DE COMPILADORES', 'location': '[ANG] 343 '}, {'time': '10:50 - 11:40', 'subject': ' PROJETO DE COMPILADORES', 'location': '[ANG] 343 '}], 'QUINTA-FEIRA': [{'time': '10:00 - 10:50', 'subject': ' PROJETO DE COMPILADORES', 'location': '[ANG] 235 '}, {'time': '10:50 - 11:40', 'subject': ' PROJETO DE COMPILADORES', 'location': '[ANG] 235 '}], 'SEXTA-FEIRA': [{'time': '17:10 - 18:00', 'subject': ' TRABALHO DE CONCLUSÃO DE CURSO II', 'location': '[ANG] 342 '}, {'time': '18:00 - 18:50', 'subject': ' TRABALHO DE CONCLUSÃO DE CURSO II', 'location': '[ANG] 342 '}]}


In [55]:
def merge_events(dictionaryVariable: dict) -> dict:
    merged_dictionary = {}

    for day, events in dictionaryVariable.items():
        merged_events = []
        i = 0
        while i < len(events):
            current_event = events[i]
            # If there's a next event and it has the same subject as the current event
            if i + 1 < len(events) and current_event['subject'] == events[i + 1]['subject']:
                next_event = events[i + 1]
                # If the end time of the current event is the start time of the next event
                if current_event['time'].split(' - ')[1] == next_event['time'].split(' - ')[0]:
                    # Merge the events
                    merged_event = {
                        'time': current_event['time'].split(' - ')[0] + ' - ' + next_event['time'].split(' - ')[1],
                        'subject': current_event['subject'],
                        'location': current_event['location'] if current_event['location'] == next_event['location'] else current_event['location'] + ', ' + next_event['location']
                    }
                    current_event = merged_event  # Set the current event to the merged event
                    i += 1  # Skip the next event

            merged_events.append(current_event)
            i += 1

        merged_dictionary[day] = merged_events

    return merged_dictionary

merge_events(schedule)

{'SEGUNDA-FEIRA': [{'time': '10:00 - 11:40',
   'subject': ' PROJETO DE COMPILADORES',
   'location': '[ANG] 343 '}],
 'QUINTA-FEIRA': [{'time': '10:00 - 11:40',
   'subject': ' PROJETO DE COMPILADORES',
   'location': '[ANG] 235 '}],
 'SEXTA-FEIRA': [{'time': '17:10 - 18:50',
   'subject': ' TRABALHO DE CONCLUSÃO DE CURSO II',
   'location': '[ANG] 342 '}]}

In [49]:
from icalendar import Calendar, Event,Alarm
from dateutil.rrule import rrule, WEEKLY, MO
from datetime import date, datetime, timedelta
import pytz
import uuid
import os
from typing import Dict, List

def createEvent(calendar: Calendar, eventName: str, startDatetime: datetime, endDatetime: datetime, timezoneStr='America/Sao_Paulo') -> Calendar:
    try:
        event = Event()
        event.add('summary', eventName)

        timezone = pytz.timezone(timezoneStr)

        event.add('dtstart', timezone.localize(startDatetime, is_dst=None))
        event.add('dtend', timezone.localize(endDatetime, is_dst=None))
        event.add('rrule', {'freq': ['WEEKLY']})

        alarm = Alarm()
        alarm.add('action', 'DISPLAY')
        alarm.add('trigger', timedelta(minutes=-15))

        event.add_component(alarm)

        calendar.add_component(event)

        return calendar
    except Exception as e:
        print(f'An error occurred while creating the event: {e}')
        return None


def rename_keys(dictionary:Dict, conversionDict:Dict)-> Dict:
    return dict([(conversionDict.get(k), v) for k, v in dictionary.items()])

def createCalendar(scheduleDict: Dict[str, List[Dict[str, str]]])-> Calendar:
  
  cal = Calendar()
  cal.add('prodid', '-//Google Inc//Google Calendar 70.9054//EN')
  cal.add('version', '2.0')
  
  # Rename keys for easier use with "rrule"
  dayConversion = {'SEGUNDA-FEIRA': 0, 'TERÇA-FEIRA':1, 'QUARTA-FEIRA':2,'QUINTA-FEIRA':3,'SEXTA-FEIRA':4,'SÁBADO':5,'DOMINGO':6}
  scheduleDict = rename_keys(scheduleDict, dayConversion)

  for day, classes in scheduleDict.items():
      for classDetails in classes:
          nextDay = rrule(freq=WEEKLY, dtstart=date.today(), byweekday=day, count=1)[0]
          startTimeString = classDetails.get('time').split('-')[0].strip()
          endTimeString = classDetails.get('time').split('-')[-1].strip()

          startTime = datetime.strptime(startTimeString, '%H:%M')
          endTime = datetime.strptime(endTimeString, '%H:%M')

          classStartDatetime = datetime(nextDay.year, nextDay.month, nextDay.day, startTime.hour, startTime.minute)
          classEndDatetime = datetime(nextDay.year, nextDay.month, nextDay.day, endTime.hour, endTime.minute)

          cal = createEvent(cal,classDetails.get('subject'),classStartDatetime, classEndDatetime)

  return cal


  
def writeCalendar(calendar: Calendar) -> str:
    # Check if the calendar instance is valid
    if not isinstance(calendar, Calendar):
        raise ValueError("Invalid calendar instance.")

    try:
        with open(calendar.ical, 'wb') as fileWriter:
            fileWriter.write(calendar.to_ical())

        return "gerou o bagulho"
    except Exception as e:
        print(f'An error occurred while writing the calendar: {e}')
        return ""
    

  

In [52]:
calendar = createCalendar(schedule)
with open(calendar.ical, 'wb') as fileWriter:
            fileWriter.write(calendar.to_ical())

AttributeError: 'Calendar' object has no attribute 'ical'